In [2]:
import pandas as pd
import numpy as np
import glob
import re
import statsmodels.api as sm
import random
import math
from scipy.optimize import *
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display,Latex,display_html
import statsmodels.formula.api as smf
import matplotlib.ticker as ticker
from numpy.linalg import inv
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

#sets the plotting style, registers pandas date converters for matplotlib, and sets the default figure size.
sns.set_style("whitegrid")
pd.plotting.register_matplotlib_converters()
# Default figure size
sns.mpl.rc('figure',figsize=(16, 6))

class analyst():
    def __init__(self):
        pass
    def myVariableName(self,variables):
        """轉出對應簡稱的代碼，ex: d1_NEM-->NEM 、 NU(-1)-->NU
        """
        names = []
        for name in variables: 
            likes=[]
            for i in d.convert['對應簡稱'].dropna():
                if i in name:
                    likes.append(i)
            maxLen=0
            moreLike=''
            for l in likes:
                if len(l) > maxLen:
                    maxLen = len(l)
                    moreLike=l
            if moreLike != '':
                names.append(moreLike)
            else:
                names.append(name)
        return names
    def smpl_ols_model(self,smpl,yloc=0):
        """smpl中第yloc個欄位是y變數"""
        y_name = smpl.columns[yloc]
        y = smpl[y_name]
        X = smpl.copy()
        X.pop(y_name)
        X = sm.add_constant(X)
        ols_model = sm.OLS(y, X)
        return ols_model
    def report_root(self,variables,date,roots):
        data = s.smpl_all(variables)
        report = pd.DataFrame(data.loc[date])
        report['root'] = roots
        names = self.myVariableName(variables)
        notes = []
        for name in names:
            notes.append(s.convert.set_index('對應簡稱').loc[name]['變數中文'])
        report['note'] = notes
        report.index.name = 'endogs'
        return report
    def pre_equ(self,smpl,endogs,solvePeriod,regNum=0,resid_into_const=True):
        """ 之後要決定residual如何處理"""
        # a:得回歸結果
        y_name = smpl.columns[0] # y固定為第一個元素
        y = smpl[y_name]
        X = smpl.copy()
        X.pop(y_name)
        X = sm.add_constant(X)
        ols_model = sm.OLS(y, X)
        y_name = ols_model.endog_names
        ols_results = ols_model.fit()
        params_df = pd.DataFrame(data=ols_results.params,columns=[y_name])
        ###
        # b: 將回歸的 y 放在 columns 並定義係數為 -1 (似等號另一邊為 0 ) 
        equDf = params_df.T.copy()
        equDf[params_df.columns[0]] = -1
        equDf.set_axis( [regNum] ,inplace=True)
        ###
        # c: 回歸中哪些變數與聯立解的內生變數無關
        exogs=[]
        for x in equDf.columns:
            if x == 'const':
                continue
            onoff = True
            for endog in endogs:
                if ( re.search('_{e}'.format(e=endog),x) or re.search('{e}\w|\w{e}'.format(e=endog),x) == None ) and endog in x:
                    onoff = False
                    break
            if onoff:
                exogs.append(x)
        # d: 將 part c的外生變數之係數*真實值與常數項(、第一期估計誤差加總後)，與內生變數併入 equDf2
        residual = ols_results.resid[solvePeriod]
        const = equDf['const'].values[0]
        if resid_into_const ==True:
            print(regNum,'resid into const')
            const = const + residual
        for exog in exogs:
            actualXparam = smpl[exog].loc[solvePeriod] * equDf[exog].values[0]
            const = const + actualXparam
        cols = []
        for c in equDf.columns:
            if c not in exogs and c != 'const':
                cols.append(c)
        equDf2 = equDf[cols].copy()
        equDf2['const'] = const
        return equDf,equDf2
    
    def equDf2_adj_dif(self,equDf2,solvePeriod,difName=None,difNum=None):
        #  difName = None : 自動搜索
        #  difName = 'd1_RMIBON'
        difs = []
        if difName==None:
            for name in equDf2.columns:
                if re.search('d\d_',name):
                    difs.append(name)
        else:
            difs.append(difName)
        for difName in difs:
            result = re.search('d(\d)_',difName)
            if result:
                difNum= int(result.group(1))
                x = difName.split(result.group())[-1]
            else:
                x = difName
                if difNum == None:
                    difNum=1
#             print(difName,x,difNum)
            data = s.smpl_all([x])
            lagValue= data.loc[solvePeriod-difNum].values * equDf2[difName].values
            equDf2['const'] = equDf2['const'].values - lagValue
            equDf2.rename(columns={difName:x},inplace=True)
            
    def equDf2_adj_lag(self,equDf2,solvePeriod):
        for name in equDf2.columns:
            result = re.search('(\w*)\(-(\d)\)',name)
            if result:
                x = result.group(1)
                lagNum = int(result.group(2))
                data = s.smpl_all([x])
                lagValue = data.loc[solvePeriod-lagNum].values * equDf2[name].values
                equDf2['const'] = equDf2['const'].values + lagValue
                equDf2.pop(name)
    

        
    def line_fit(self,ols_results=False,dataFit=None,seriesActual=None,resid_ybound=True,name='untitled',
                 xinterval=4 ,savePath=None):
        """
        1.輸入真實值跟預測值序列:
        a.line_fit(ols_results=False, dataFit=predictDf['NU'], seriesActual=data['NU'], resid_ybound=True, name='NU')
        2.輸入ols_results跟真實值序列
        a.line_fit(ols_results=ols_results, dataFit=None, seriesActual=data['NU'], resid_ybound=True, name='untitled')
        """
        if ols_results:
            t = ols_results.resid.index.to_timestamp()
            dataResid = ols_results.resid.values
            dataFit = ols_results.fittedvalues.values
        else:
#             t = seriesActual.index.to_timestamp()
            t=seriesActual.index.strftime('%fQ%q')
            dataResid = seriesActual-dataFit
        lineZero = [0]*len(t)
        fig, ax1 = plt.subplots()
        ax1.plot(t, dataResid, color='green',label='residual',marker='o',markersize=3.5)
        ax1.plot(t,lineZero,color='black',linestyle=':')
        ax1.set_xlabel(xlabel='time(Q)',size=13,color='black',rotation=10)
        ax1.set_ylabel(ylabel='resid',size=20,color='green',rotation=0,position=(0,0.95))
        ax1.tick_params(axis='x', labelcolor='black',labelrotation=10,labelsize=13)
        ax1.tick_params(axis='y', labelcolor='green',labelsize=13)

        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        ax2.plot(t, dataFit, color='r',marker='o',markersize=2)
        if type(seriesActual) == pd.core.series.Series:
            r2 = r2_score(seriesActual,dataFit)
            rmse = np.sqrt(mean_squared_error(seriesActual,dataFit))
#             ax1.set_title(label='R-squared=%.3f ,rmse = %.3f'%(r2,rmse),fontdict={'fontsize':20})
            ax1.set_title(label='rmse = %.3f'%rmse,fontdict={'fontsize':20})
            ax2.plot(t, seriesActual, color='b',marker='o',markersize=2)
        if resid_ybound:
            # 取得上下界的值
            y2min,y2max = ax2.get_ybound()
            up_bound = (y2max-y2min)/2
            # 設定顯示哪些刻度，此例切8等分
            ax2.set_yticks([x for x in np.arange(y2min,y2max,up_bound*2/8)])
            # 設定左y軸全距 = 右y軸全距
            ax1.set_ybound(-up_bound,up_bound)
            ax1.set_yticks(np.arange(-up_bound,up_bound,up_bound*2/8)) # 誤差刻度也是切8等分
        ax2.tick_params(axis='y', labelcolor='b',labelsize=13)
        ax2.legend(labels=['fitted_'+name,'actual_'+name,'resid'],loc=0,fontsize=20,shadow=True)
        ax2.axvline(x='19Q1',linestyle='-.',color='black') #### 畫垂直線
        
        if xinterval:
            ax1.xaxis.set_major_locator(ticker.MultipleLocator(xinterval))
        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        if savePath:
            plt.savefig(fname=savePath)
        plt.show()
    def line_fit2(self,ols_results=False,dataFit=None,seriesActual=None,resid_ybound=True,name='untitled',
                 xinterval=4 ,savePath=None):
        t=seriesActual.index.strftime('%fQ%q')
        dataResid = seriesActual-dataFit
        lineZero = [0]*len(t)
        fig, ax1 = plt.subplots()
        ax1.plot(t, dataResid, color='green',label='residual',marker='o',markersize=3.5)
        ax1.plot(t,lineZero,color='black',linestyle=':')
        ax1.set_xlabel(xlabel='time(Q)',size=13,color='black',rotation=10)
        ax1.set_ylabel(ylabel='resid',size=20,color='green',rotation=0,position=(0,0.95))
        ax1.tick_params(axis='x', labelcolor='black',labelrotation=10,labelsize=13)
        ax1.tick_params(axis='y', labelcolor='green',labelsize=13)

        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        ax2.plot(t, dataFit, color='r',marker='o',markersize=2)
        if type(seriesActual) == pd.core.series.Series:
            r2 = r2_score(seriesActual,dataFit)
            train_rmse = np.sqrt(mean_squared_error(seriesActual.loc['1995Q1':'2018Q4'],dataFit.loc['1995Q1':'2018Q4']))
            test_rmse = np.sqrt(mean_squared_error(seriesActual.loc['2019Q1':'2020Q4'],dataFit.loc['2019Q1':'2020Q4']))
        #             ax1.set_title(label='R-squared=%.3f ,rmse = %.3f'%(r2,rmse),fontdict={'fontsize':20})
            ax1.set_title(label='train RMSE: %.3f,  test RMSE: %.3f'%(train_rmse,test_rmse),fontdict={'fontsize':20})
            ax2.plot(t, seriesActual, color='b',marker='o',markersize=2)
        if resid_ybound:
            # 取得上下界的值
            y2min,y2max = ax2.get_ybound()
            up_bound = (y2max-y2min)/2
            # 設定顯示哪些刻度，此例切8等分
            ax2.set_yticks([x for x in np.arange(y2min,y2max,up_bound*2/8)])
            # 設定左y軸全距 = 右y軸全距
            ax1.set_ybound(-up_bound,up_bound)
            ax1.set_yticks(np.arange(-up_bound,up_bound,up_bound*2/8)) # 誤差刻度也是切8等分
        ax2.tick_params(axis='y', labelcolor='b',labelsize=13)
        ax2.legend(labels=['fitted_'+name,'actual_'+name,'resid'],loc=0,fontsize=20,shadow=True)
        ax2.axvline(x='19Q1',linestyle='-.',color='black') #### 畫垂直線

        if xinterval:
            ax1.xaxis.set_major_locator(ticker.MultipleLocator(xinterval))
        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        if savePath:
            plt.savefig(fname=savePath)
        return fig,ax1,ax2
    
    def line_resid(self,dataFit=None,seriesActual=None,xinterval=None,savePath=None):
        t=seriesActual.index.strftime('%fQ%q')
        residual = dataFit - seriesActual
        percent_resid = np.square((seriesActual - dataFit)/seriesActual)*100
        fig, ax1 = plt.subplots()
        ax1.plot(t, residual, color='green',label='residual',marker='o',markersize=3.5)
        # ax1.plot(t, percent_resid, color='green',label='residual',marker='o',markersize=3.5)
        ax1.set_xlabel(xlabel='time(Q)',size=13,color='black',rotation=10)
        ax1.set_ylabel(ylabel='resid',size=20,color='green',rotation=0,position=(0,0.95))
        ax1.tick_params(axis='x', labelcolor='black',labelrotation=10,labelsize=13)
        ax1.tick_params(axis='y', labelcolor='green',labelsize=13)
        if xinterval:
            ax1.xaxis.set_major_locator(ticker.MultipleLocator(xinterval))
        fig.tight_layout()
        if savePath:
            plt.savefig(fname=savePath)
        plt.show()
        return percent_resid
a = analyst()

def show_tex_reg(endogsNameList,variable_2D):
    for variable in variable_2D:
        text = r'$ \color{brown}{\textit{'+variable[0] + r'}}=\beta_0'
        for it,x in enumerate(variable[1:]):
            it = it + 1
            if x in {e for e in endogsNameList}:
                te = r'+\beta_'+str(it)+r'\space\color{brown}{\textit{'+x+'}}'
            else:
                te = r'+\beta_{}\space{}'.format(it,r'{\textrm{'+x+'}}')
            text= text+te
        text=text+r'+\epsilon_t $ '
    #     text= text.replace('log_','log\_').replace('pch_','pch\_').replace('d1_','d1\_')
        display(Latex(text))
    #     alltext = alltext +text
    # display(Latex(alltext))

def show_df(dfs, names=[]):
    count = 0
    maxTables = 6
    if not names:
        names = [x for x in range(len(dfs))]
    html_str = ''
    html_th = ''
    html_td = ''
    for df, name in zip(dfs, names):
        if count <= (maxTables):
            html_th += (''.join(f'<th style="text-align:center">{name}</th>'))
            html_td += (''.join(f'<td style="vertical-align:top"> {df.to_html(index=False)}</td>'))
            count += 1
        else:
            html_str += f'<tr>{html_th}</tr><tr>{html_td}</tr>'
            html_th = f'<th style="text-align:center">{name}</th>'
            html_td = f'<td style="vertical-align:top"> {df.to_html(index=False)}</td>'
            count = 0
    if count != 0:
        html_str += f'<tr>{html_th}</tr><tr>{html_td}</tr>'
    html_str += f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)

def stepwise_selected(data,response,backward=True,upbound=None):
    """
    backward=False : only foreward
    backward=True : backward by remove lowest contribution selected  # 找出最高分數(adjusted R-squared)的組合
    score: 預設用adjusted R-squared 來決定變數的進出，之後可以新增選項(AIC,F test等等)
    """    
    remaining = [x for x in data.columns]
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    i=0
    while remaining and current_score == best_new_score:
        i=i+1
        # foreward部分起始位置
        scores_with_candidates = []
        for candidate in remaining:
            formula = "Q('{}') ~ {} + 1".format(response,
                            " + ".join(["Q('"+x+"')" for x in selected] + ["Q('"+candidate+"')"]))
            # ex: NU ~ Q('NU(-1)') + Q('Q3') + 1 ，解釋變數為 NU(-1)、Q3，有截距項。 加了Q('x')確保正確抓出x此變數
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort() #依據score排序
        best_new_score, best_candidate = scores_with_candidates.pop() #存出最高分的
#         print(i,best_new_score, best_candidate)
        if current_score <= best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
        # foreward部分結束位置
            # backward : 當前當選的變數群(x變數群)，用 full model - reduce model的 adjusted R-squared 得各變數的貢獻度，
            #             若最低貢獻不是最近一次選進的變數則丟回候選池。
            # 通常情況上面每圈選進一個新變數，他的貢獻(提升模型adjusted R-squared的程度)應該會比其他x變數來得低，
            # 但如果最低貢獻不是最近一次選進的變數時，則將該變數剔除x變數群。
            if backward:
                if i <=2: # 有三個解釋變數再開始向後
                    continue
                contribute_with_electeds = []
                # ex: 已選進了三個變數，selected = [a,b,c]， current_score為此三變數回歸的 adjusted R-squared
                for j,elected in enumerate(selected):
                    # j=0, elected = a 時
                    contribute_elected = elected
                    otherSelected = selected.copy()
                    otherSelected.remove(elected)
                    formula = "Q('{}') ~ {} + 1".format(response,
                                " + ".join(["Q('"+x+"')" for x in otherSelected]))
                    # formula = NU ~ Q('b') + Q('c') + 1
                    score2 = smf.ols(formula, data).fit().rsquared_adj
                    contribute_score = current_score - score2 #得到 a的對adjusted R-squared貢獻
                    contribute_with_electeds.append((contribute_score,contribute_elected))
                contribute_with_electeds.sort() # 得 a,b,c的貢獻
                lowest_score_elected = contribute_with_electeds[0][1] # 假設 b最低，則lowest_score_elected=b
                if lowest_score_elected != selected[-1]: # b != c
#                     print('第 {} round:剔除{}'.format(i,lowest_score_elected)) # 剔除 b
                    remaining.append(lowest_score_elected) # b丟回候選池
                    selected.remove(lowest_score_elected) # 將b從x變數群移除
        else:               
            print('第{} round end \n 選進{}個變數'.format(i,len(selected)))
        if not remaining:
            print('第{} round end \n 選進所有變數({}個)'.format(i,len(selected)))
        if upbound:
            if len(selected) >= upbound:
                break
    return selected

def show_ols_coef(X ,y,indexName=None):
    a = analyst()
    # X = x_train[S],y = y_train
    ori_s = a.myVariableName(X.columns)
    
    X = sm.add_constant(X)
    ols_model = sm.OLS(y, X)
    ols_results = ols_model.fit()
    # ols_results.summary()
    names = [] 
    for x in ori_s:
        if x in {'Q1','Q2','Q3','Q4'}:
            name = x
        else:
            name = s.convert.set_index('對應簡稱').loc[x]['對應中文']
        names.append(name)
    paramdf = pd.DataFrame(data = {'exog':ols_results.model.exog_names,
                                   'coef':ols_results.params.values,
                                   't':ols_results.tvalues.values,
                                   'p-value':ols_results.pvalues.values, 
                                  },index = ['const'] + names)
    pd.options.display.float_format = "{:,.4f}".format
    if indexName:
        paramdf.index.name = indexName
    return paramdf

def lasso_stepwise(nadata,       # 放入未 partial out Lag.Y 的 nadata
                    endog,      # 內生變數在nadata的欄位名
                    trainEndPeriod,   # train(樣本內預測期間)的最後一期 ex: '2018Q4'
                    CV_n,      # LassoCV的TimeSeriesSplit(n_splits= "n")
                    alphaRange, # 檢驗alpha的範圍: np.logspace(-1,3,num=100)
                    partialOutAR1 = True, # 是否要partial out AR(1) 的資料丟入Lasso
                    showFitR2 =True,  # 顯示此Lasso模型在 test 的表現
                    showShrink=True ): # 顯示Lasso係數收縮圖
    """return L,S,ols_results 說明:
    L: lasso selected variables
    S: 跑 endog 對 L 變數在最大樣本期間的逐步迴歸，存出 S 變數list
    ols_results: 存出 endog 對 S 迴歸的資訊
    """
    print('#'*100+f'\nendog= {endog} ,cv=TimeSeriesSplit(n_splits={CV_n})---------------------',end='')
    data = nadata.dropna()
    N = data.shape[0]
    if partialOutAR1:
        x1 = data[endog+'(-1)'].values.reshape(N,1)
        M1 = np.eye(N)-x1@inv(x1.T@x1)@x1.T # M1 = I-X1(X1'X1)^{-1}X1'
        y = M1@data[endog].values # M1y
        x = data.copy()
        x = M1@x
        x.drop(columns=[endog,'d1_'+endog,endog+'(-1)'],axis=0,inplace=True)
    else:
        print( 'not partial out AR(1) !')
        y = data[endog].values
        x = data.copy()
        x.drop(columns=[endog,'d1_'+endog],axis=0,inplace=True)
    x.index = data.index
    trainEndPeriod = pd.Period(trainEndPeriod,freq='Q-DEC')
    x_train,x_test = x.loc[:trainEndPeriod],x.loc[trainEndPeriod+1:]
    y_train,y_test = np.split(y,[x.shape[0]-x_test.shape[0]])
    #     print(x.shape , y.shape,x_train.shape,x_test.shape,y_train.shape,y_test.shape)
    print('train last period:',x_train.index[-1],','
          'test last period',x_test.index[-1])
    alphas_grid = np.logspace(alphaRange[0], alphaRange[1], num = 200) # 範圍放比較大的，故使用200個點
    lasso_cv = linear_model.LassoCV(alphas = alphas_grid, fit_intercept = True, normalize = True, max_iter = 100000,
                                    tol = 0.0001, cv = TimeSeriesSplit(n_splits=CV_n))
    lasso_cv_fitted = lasso_cv.fit(x_train, y_train)
    print('best alpha: ',lasso_cv_fitted.alpha_)
    best_alpha = lasso_cv_fitted.alpha_
    k = lasso_cv_fitted.coef_[lasso_cv_fitted.coef_!=0].shape[0]
    print('lasso選進',k,'個')
    L = x_train.columns[lasso_cv_fitted.coef_!=0]
    print('lasso選出的變數集合L:',L)
    fig, ax1 = plt.subplots()
    ax1.plot(lasso_cv_fitted.alphas_,lasso_cv_fitted.mse_path_, ':')
    ax1.plot(lasso_cv_fitted.alphas_, lasso_cv_fitted.mse_path_.mean(axis = -1),
             color='green',marker='o',markersize=3.5)
    ax1.set_xlabel(xlabel='alpha',size=13,color='black',rotation=10)
    ax1.set_ylabel(ylabel='mse',size=20,color='black',rotation=10,position=(0,1))
    plt.axvline(lasso_cv_fitted.alpha_, linestyle = '--', color = 'k', label = 'alpha: CV estimate')
    ax1.set_xscale('log')
    plt.show()

    y_hat_train = lasso_cv_fitted.predict(x_train)
    y_hat_test = lasso_cv_fitted.predict(x_test)
    if showFitR2:
        print('Performance on Training Data')
        print('  + RMSE: %.8f' % np.sqrt(mean_squared_error(y_train, y_hat_train)))
        # print('  + mean squared error: %.4f' % mean_squared_error(y_train, y_hat_train))
        print('  + R squares: %.8f' % r2_score(y_train, y_hat_train))
        print('Performance on Testing Data')
        print('  + RMSE: %.8f' % np.sqrt(mean_squared_error(y_test, y_hat_test)))
        print('  + R squares: %.8f' % r2_score(y_test, y_hat_test))
        print('\n')
    if showShrink:
        print(f'\nendog={endog}--coef shrinkage:')
        alpha100 = []
        for i,num in enumerate(alphas_grid):
            alpha100.append((num,num+0.000000000001))
        coefs = []
        klist = []
        for a in alpha100:
            lasso_cv = linear_model.LassoCV(alphas = a, fit_intercept = True, normalize = True, max_iter = 100000,
                                        tol = 0.001, cv = TimeSeriesSplit(n_splits=CV_n))
            lasso_cv_fitted = lasso_cv.fit(x_train,y_train)
            coefs.append(lasso_cv_fitted.coef_)
            k = lasso_cv_fitted.coef_[lasso_cv_fitted.coef_!=0].shape[0]
            klist.append(k)

        fig, ax1 = plt.subplots()
        ax1.plot([a[0] for a in alpha100],coefs)
        # ax1.set_ybound(-100,100)
        plt.axvline(best_alpha, linestyle = '--', color = 'k', label = 'alpha: CV estimate')
        ax1.set_xscale('log')
        plt.show()
    
    smpl = x_train[L].copy()
    smpl[endog] = y_train.copy()
    S = stepwise_selected(data=smpl,response=endog,backward=True)
    print('stepwise選出的變數集合S:\n',S,'\n')

    # # "內生變數落後期"放進回歸自變數
#     X = nadata[S].loc[:trainEndPeriod].dropna()
#     X = sm.add_constant(X)
#     ols_results = sm.OLS(nadata[endog].loc[X.index].dropna(),X).fit()
    # #     ols_results.save(f'../reg_pickle/{datetime.datetime.today().date()}{endog}.pickle')
    return L,S

mutiFunc=['GDP = CF + CO + IBF + IG + IPC + J + CG + X - M',
          'NE = (1-0.01*NU)*NF',
          'PDT = GDP/NE',
          'ULC = PWM/PDT',
         ]
# exogsLin = ['CG','CHINAGDP','EJAP','FFR','FIA',
#     'GOVSUB','IG','USAGNP','JAPGDP','IPXUSA',
#     'IR','KGDEBT','POILBRE','POP','STOCKTRADE', # 'GOVSURRP','IPC','IPXJAP','IRCUS','KF','RTAXCUM','WPX'
#     'IRI','IRC']  #此列的變數為新增的外生
def show_tex_reg2(endogsNameList,exogsNameList,regressors2D,Lin=True,plusTextList=None):
    endogsNameList.extend(['GDP','NE','PDT','ULC'])
    variable_2D = []
    for i in regressors2D:
        variable_2D.append(i)
    for i,regressors in enumerate(variable_2D):
        # 秀每條設定的回歸y,x代碼(regResults2)
        y = endogsNameList[i]
        text = r'$ {}.\space'.format(i+1) + r'\color{brown}{\textit{'+ y + r'}}= f(' # y=f(
        for it,x in enumerate(regressors):
            ori_x = a.myVariableName([x])[0]
            if ori_x in set(endogsNameList):
                te = r'\space\color{brown}{\textit{'+x+'}}\space,'
            elif Lin and ori_x in set(exogsNameList):
                te = r'\space\color{blue}{\textit{'+x+'}}\space,'
            else:
                te = r'\space{}'.format(r'{\textrm{'+x+'}}\space,')
            text= text+te
        text=text+r'\space) $ '        
#         text= text.replace('log_','log\_').replace('pch_','pch\_').replace('d1_','d1\_')
        display(Latex(text))
    if plusTextList:
        total_num = len(regressors2D)
        for i,text in enumerate(plusTextList):
            text = r'$ {}.\space'.format(total_num+i+1)+'{'+text+r'}$'
            display(Latex(text))
            
def show_tex_reg3(endogsNameList,exogsNameList,regressors2D,Lin=True,plusTextList=None,greenGroup=None):
    regLin = {
        'SALES': ['GDP','AR(1)=0.76'],
        'GDPMFG':['GDP','TECH','AR(1)=0.62'],
        'TECH':['d1_POILSAR','log(I+I(-1)+I(-2)+I(-3)+I(-4)','AR(1)=0.85'],
        'log_CF':['log_CF(-4)','GDP','log_POP','log_PSTOCK','CPI','AR(1)=0.002'],
        'log_CO':['log_CO(-4)','IRC-pch_CPIZF','log_PSTOCK','pch_LOAN','Q1','Q3'],
        'IBF':['IBF(-4)','DEP(-1)+DEP(-2)','d1_KF','SALES/GDP','IRI(-1)','LOAN(-1)','POILSAR','STOCKTRADE',
              'BONDTRADE','Q1','Q2','Q3'],
        'J':['J(-4)','IRI(-1)-pch_CPI(-1)','I(-1)','Q3','AR(1)=0.2'],
        'DEP':['GDP*DEP(-1)/GDP(-1)','I(-3)','WPI','CPI'],
        'TAXD\$':['GDP*TAXD\$(-4)/GDP(-4)','PWM','AR(1)=0.22'],
        'TAXID\$':['TD*TAXID\$(-4)/TD(-4)','SALES','AR(1)=0.33'],
        'RKGDBT':['RKGDBT(-1)','GOVSURRP\$'],
        'M':['M(-4)','TD','PM','WPX','Q1','Q2','AR(1)=0.58'],
        'EX':['EX(-4)','EROC','GDPMFG','IGNPUSA','CHINAGDP'],
        'NF':['POP*NF(-4)/POP(-4)','pch_GDP','pch_PWM','AR(1)=0.64'],
        'NU':['NU(-4)','SALES/GDP','CHINAGDP','D2003','AR(1)=0.82'],
        'EROC':['EROC(-1)','PSTOCK','RKGDBT','AFR\$/GNP\$','(IRC-IRCUS)','EJAP','AR(1)=0.37'],
        'MON\$':['RMIBON','d1_PSTOCK','d1_GDP','MON\$(-1)','pch_CPI','Q1','Q2','Q3'],
        'M2':['M2(-1)','GDP','(IRC-IRCUS)','d1_STOCKTRADE','Q1','Q2'],
        'ADRESERVE\$':['GDP\$','DEPOSIT','d1_RMIBON','AR(1)=0.81'], ###
        'd1_AFR\$/GDP\$':['d1_TB\$+d1_FA\$+d1_FIA\$/GDP\$','EROC','FFR','d1_AFR\$(-1)','pch_MON\$'],
        'RMIBON':['IR','pch_CPI','log_PSTOCK','ADRESERVE\$'], ###
        'IRC':['IR','RMIBON','D2003','AR(1)=0.86'], ###
        'IRI':['FFR','RMIBON','D2003','IR','AR(1)=0.87'],
        'log_DEPOSIT':['log_GDP','d1_log_GDP','d1_(IRC-pch_CPIZF)','pch_STOCKTRADE','AR(1)=0.57'],
        'pch_LOAN':['pch_MON\$','IRI(-4)','pch_IG','pch_(IBF+CO)','AR(1)=0.96'],
        'ADBNF\$':['MON\$','RMIBON','PSTOCK','LOAN','AR(1)=0.92'], ###
        'BONDTRADE':['IRC','LOAN','PSTOCK'],
        'FA\$':['d1_EROC','d1_FFR','PSTOCK','AR(1)=0.58'],
        'PSTOCK':['STOCKTRADE','(IRI-IRC)','POILSAR','SALES','BONDTRADE','MON\$'],
        'TMUIA\$\$':['POILSAR','WPX','IPXJAP','AR(1)=0.98'],
        'PM':['(1+0.01*RTAXCUM)*TMUIA\$\$*EROC','POILSAR','IPXUSA','Q1','Q2','Q3','AR(1)=0.71'],
        'log_PWM':['log_PWM(-4)','log_CPI','NU','log_PDT','Q1','AR(1)=0.52'],
        'WPI':['d1_V90','pch_ULC','PGDP','PM','WPX','AR(1)=0.97'],
        'CPI':['(IRI(-1)-IRC(-1))','WPI','CPIZF','D2000','pch_TAXID\$','Q1','Q2','Q3','AR(1)=0.37'],
        'd1_CPIZF':['d1_RMIBON','NU','log_M2','AR(1)=-0.27'],
        'PCF':['PCF(-4)','CPI','d1_CPI','Q2','AR(1)=0.74'],
        'PCO':['PCO(-1)','D2000','WPI','AR(1)=0.23'],
        'PCG':['CPI','PCG(-4)'],
        'PIG':['WPI','Q1','Q2','AR(1)=0.98'],
        'PIPC':['PIPC(-4)','WPI','Q2','Q3','Q4','AR(1)=0.86'],
        'PIBF':['PIBF(-4)','WPI','Q1','AR(1)=0.94'],
        'PX':['PX(-4)','WPI','Q3','AR(1)=0.72'],
        'PFIA':['PFIA(-1)','CPI','AR(1)=0.87'],
        'PJ':['PJ(-1)','WPI']
    }
    endogsNameList.extend(['GDP','NE','PDT','ULC'])
    variable_2D = []
    for i in regressors2D:
        variable_2D.append(i)
    for i,regressors in enumerate(variable_2D):
        #part: 秀每條設定的回歸y,x代碼(regResults2)
        y = endogsNameList[i]
        ori_y = a.myVariableName([y])[0]
        # 找出對應林迴歸中相同的regressor
        for yLin in regLin.keys():
            if y == a.myVariableName([yLin])[0]: # 林的內生可能不是用level值
                sameRegressorSet = set(a.myVariableName(regLin[yLin]))&set(a.myVariableName(regressors))
                break
        text = r'$ {}.\space'.format(i+1) + r'{\textit{'+ y + r'}}= f(' # y=f(
        for it,x in enumerate(regressors):
            ori_x = a.myVariableName([x])[0]
#             if ori_x in set(endogsNameList):
#                 te = r'\space\color{brown}{\textit{'+x+'}}\space,'
            if Lin and ori_x in set(exogsNameList):
                te = r'\space\color{blue}{\textit{'+x+'}}\space,'
            elif ori_x in sameRegressorSet:
                te = r'\space\color{brown}{\textit{'+x+'}}\space,'
            else:
                te = r'\space{}'.format(r'{\textrm{'+x+'}}\space,')
            if greenGroup:
                if ori_x in greenGroup:
                    te = r'\space\color{green}{\textit{'+x+'}}\space,'
            text= text+te
        text=text+r'\space) $ '        
#         text= text.replace('log_','log\_').replace('pch_','pch\_').replace('d1_','d1\_')
        display(Latex(text))
        #part: 秀regResults2代碼對應中文
        ignore = {'GDP','M2','CHINAGDP','USAGNP','JAPGDP','J','M'}
        text2 = '$ \quad$說明:'
        text2 += ori_y+'~'+s.convert.set_index('對應簡稱').loc[ori_y]['對應中文']+',   '
        for it,x in enumerate(regressors):
            ori_x = a.myVariableName([x])[0]
            if ori_x in set(s.convert['對應簡稱']) and ori_x not in ignore:
                text2 += ori_x+'~'+s.convert.set_index('對應簡稱').loc[ori_x]['對應中文']+',   '
#         text2 +='$'
        display(Latex(text2))
        #part: 秀林建甫的迴歸
        for yLin in regLin.keys(): 
            if y == a.myVariableName([yLin])[0]: # 林的內生可能不是用level值，故要轉換名稱
                text3 = r'$ \quad林:\textrm{'+yLin+'}=f('
                for x in regLin[yLin]:
                    ori_x = a.myVariableName([x])[0]
                    if Lin and ori_x in set(exogsNameList):
                        text3 += r'\space\color{blue}{\textit{'+x+'}}\space,'
                    elif ori_x in sameRegressorSet:
                        text3 += r'\space\color{brown}{\textit{'+x+'}}\space,'
                    else:
                        text3+=r'\space\textrm{'+x+'},'
                text3 +=')$'
                display(Latex(text3))
                #part: 秀其代碼對應中文
                text4 = '$ \quad$說明:'
                for it,xLin in enumerate(regLin[yLin]):
                    ori_x = a.myVariableName([xLin])[0]
                    if ori_x in set(s.convert['對應簡稱']) and ori_x not in ignore:
                        text4 += ori_x+'~'+s.convert.set_index('對應簡稱').loc[ori_x]['對應中文']+',   '
                display(Latex(text4))
                break
    #part: 秀額外的方程式
    if plusTextList:
        total_num = len(regressors2D)
        for i,text in enumerate(plusTextList):
            text = r'$ {}.\space'.format(total_num+i+1)+'{'+text+r'}$'
            display(Latex(text))
            
####################################################################################
def macro_model(data, #(nadata) 放入每個變數最大樣本期間的DataFrame
                endogs, # 內生變數名稱 list
                regResults2, # 有迴歸變數組合的DataFrame
                regressor_column, #(final) regResults2中解釋變數的欄位名稱
                trainStart, # ex: '1995Q1'
                trainEnd, # '2018Q4'
                testEnd): 
    partialout_on =True
    trainStartPeriod = pd.Period(trainStart, 'Q-DEC')
    trainEndPeriod = pd.Period(trainEnd, 'Q-DEC')
#     endogs = list(regResults2.index)+['GDP','NE','PDT','ULC']
    print('所有內生:',endogs)
    ####### train期間內每條回歸的smpl DataFrame、係數方程式 DataFrame、外生變數名 list #########
    trainPeriodRange = pd.period_range(start = trainStartPeriod,end =trainEndPeriod)
    train_smpls =[] # 每條迴歸的dataFrame
    train_ols_results=[] # 回頭檢查回歸用
    train_equDfs = [] # 每條迴歸的coef表示成函數 f(y,y(-1),exogs)=0 的dataFrame
    all_exogs = [] # 存出所有外生變數名稱
    # variable_2D = [ [y ,y(-1) , exogs],[y ,y(-1) , exogs],... ]
    variable_2D=[]
    for row in regResults2.iloc:
        if partialout_on:
            variable_2D.append([row.name,f'{row.name}(-1)']+row[regressor_column])
        else:
            variable_2D.append([row.name]+row[regressor_column])
    # regNum,variables = 0,variable_2D[0]
    for regNum,variables in enumerate(variable_2D):
    #     smpl = s.smpl_all(variables).loc[:trainEndPeriod] #最大樣本期間
        smpl = data[variables].loc[:trainEndPeriod].dropna() #最大樣本期間
        train_smpls.append(smpl)
        yName = smpl.columns[0]
        y = smpl[yName] # y固定為第一個元素
        X = smpl.copy()
        X.pop(yName)
        X = sm.add_constant(X)
        ols_results = sm.OLS(y, X).fit()
        train_ols_results.append(ols_results)
        params_df = pd.DataFrame(data=ols_results.params,columns=[yName])
        ###
        # part b: 將回歸的 y 放在 columns 並定義係數為 -1 (似等號另一邊為 0 ) 
        equDf = params_df.T.copy()
        equDf[params_df.columns[0]] = -1
        equDf.set_axis( [regNum] ,inplace=True)
        train_equDfs.append(equDf)
        # c: 存出回歸中哪些變數與聯立解的內生變數無關的list，方便下面自動執行外生係數乘
        exogs=[]
        for i,name in enumerate( a.myVariableName(variables) ):
            if name not in set(endogs):
                exogs.append(variables[i])
    #     print(regResults2.iloc[regNum].name,'\n',regResults2[regressor_column].iloc[regNum])
    #     print(exogs,'\n#################################################')
        all_exogs.append(exogs)

    ############## 2. train期間聯立解存到 predictDf ################
    
    # 新增聯立解用的一條方程式:
    # CF + CO + IBF + IG + IPC + J + CG + X - M = GDP
    # CF + CO + IBF + J + X - M - GDP + const = 0  ( const=IG+IPC+CG )
    gdpdf = s.smpl_all(['CF','CO','IBF','IG','IPC','J','CG','EX','M','GDP'])
    gdpequs = pd.DataFrame([[1,1,1,1,1,-1,-1,1]],columns=['CF','CO','IBF','J','EX','M','GDP','const'],index=gdpdf.index)
    gdpequs['const'] = gdpdf[['IG','IPC','CG']].T.sum() # 每期的coef跟const
    # 已確認相等(gdpdf[gdpequs.columns[:-1]]*gdpequs[gdpequs.columns[:-1]]).T.sum()*(-1) == gdpequs['const']

    root_rowdata=[]
    root_indexs=[]
    endogDf = data[endogs]
    # 從 train第一期聯立解到最後一期的rowdata存進 predictDf DataFrame
    for period in trainPeriodRange:
        # period = trainPeriodRange[0]
        print(period,end=',')
        allEqu = pd.DataFrame()
        for regNum,variables in enumerate(variable_2D):
    #         print(regNum,variables)
            smpl,equDf,exogs = train_smpls[regNum],train_equDfs[regNum],all_exogs[regNum]
            # part d: 將 part c的外生變數之係數*真實值與常數項 (估計誤差不納入計算)，與內生變數併入 equDf2
            const = equDf['const'].values[0] + (smpl[exogs].loc[period]*equDf[exogs]).T.sum()  # float
            ### 納入residual的話聯立解會完全等於真實值，用來檢查過程有無錯誤
    #         residual = train_ols_results[regNum].resid.loc[period] # float
    #         const = const + residual
    #         print(regNum,'resid into const')
            cols = []
            for c in equDf.columns:
                if c not in exogs and c != 'const':
                    cols.append(c)
            equDf2 = equDf[cols].copy()
            equDf2['const'] = const # equDf2 = 內生變數相關 (例如:年增率,差分,lag期)的係數跟方程式的常數項
    #         print('外生乘係數加進常數項後剩的columns:',equDf2.columns)
            # part e:轉換成當期函數

            # 找出差分項，再將lag期真實值乘係數加進const，ex: d1_xxxxx
            # def equDf2_adj_dif(self,equDf2,solvePeriod,difName=None,difNum=None) 
            solvePeriod,difName,difNum=period,None,None #  difName = 'd1_RMIBON', difName = None : 自動搜索
            difs = []
            if difName==None: #找出差分項的名稱
                for name in equDf2.columns:
                    if re.search('d\d_',name):
                        difs.append(name)
            else:
                difs.append(difName)
    #         print(f'差分項:{difs}')
            for difName in difs:
                result = re.search('d(\d)_',difName)
                if result:
                    difNum= int(result.group(1))
                    ori_x = difName.split(result.group())[-1]
                else: #  其他非照規則(d1_)的自定義差分項
                    ori_x = difName
                    if difNum == None:
                        difNum=1
    #             print(ori_x,difNum,end=',')
                df = s.smpl_all([ori_x])
                lagValue= df.loc[solvePeriod-difNum].values * equDf2[difName].values
                equDf2['const'] = equDf2['const'].values - lagValue
                if ori_x in equDf2.columns:
    #                 print(f'將同為{ori_x}的係數相加')
                    equDf2[ori_x] = equDf2[ori_x].values + equDf2[difName].values
                    equDf2.pop(difName)
                else:
            #         print(f'原{difName}改名為{ori_x}')
                    equDf2.rename(columns={difName:ori_x},inplace=True)
            # 找出lag項真實值乘係數加進const 
            # def equDf2_adj_lag(self,equDf2,solvePeriod):
            solvePeriod= period
    #         print('\nlag項:',end=' ')
            for name in equDf2.columns:
                result = re.search('(\w*)\(-(\d)\)',name)
                if result:
                    x = result.group(1)
                    lagNum = int(result.group(2))
    #                 print(x,lagNum,end=',')
                    df = s.smpl_all([x])
                    lagValue = df.loc[solvePeriod-lagNum].values * equDf2[name].values
                    equDf2['const'] = equDf2['const'].values + lagValue
                    equDf2.pop(name)

            # 找出年增率處理成當期
    #         print('\n年增率項:',end=' ')
            for name in equDf2.columns:
                if 'pch_' in name:
                    ori_x = name.split('pch_')[1]
    #                 print(name,'->',ori_x,end=', ')
                    equDf2['const'] = equDf2['const'].values-equDf2[name]
                    df= s.smpl_all([ori_x])
                    equDf2[name] = equDf2[name].values/df.loc[solvePeriod-4].values # coef/y(-4)
                    # 若該年增率的內生變數剛好是這條回歸的y時，當pch_y轉成y後的係數coef/y(-4)可與係數為-1的y係數相加
                    if ori_x in equDf2.columns:
    #                     print(f'將同為{ori_x}的係數相加')
                        equDf2[ori_x] = equDf2[ori_x].values + equDf2[name].values
                        equDf2.pop(name)
                    else:
    #                     print(f'原{name}改名為{ori_x}')
                        equDf2.rename(columns={name:ori_x},inplace=True)
    #         print(f'\n處理完的columns:{equDf2.columns}')
            allEqu=pd.concat([allEqu,equDf2])
        # equ: CF + CO + IBF + J + X - M - GDP + const = 0  ( const=IG+IPC+CG )
        equ = pd.DataFrame(gdpequs.loc[period]).T
        allEqu = pd.concat([allEqu,equ])

        allEqu = allEqu[endogs+['const']]
        # part f: 猜該期真實值 解聯立
        guess = endogDf.loc[period].values
        ###
        def system(z,allEqu=allEqu): 
        #     if allEqu.columns[-1] != 'const':
        #         n = list(allEqu.columns).index('const')
        #         allEqu = allEqu[list(allEqu.columns[:n])+list(allEqu.columns[n+1:])+['const']]
            colNum={} # colNum[內生變數名] = 內生變數的欄位順序
            for i,col in enumerate(allEqu.columns):
                colNum[col] = i
            f= np.zeros(allEqu.shape[0]+3) # 3個額外的函數就加3
            # 適用於每條方程式皆係數*value加常數項
            for i,equ in enumerate(allEqu.iloc):
                f[i] = (equ[:-1]*z).sum()+ equ['const']
            # 函數裡的變數剛好全是內生變數時(都有在allEqu)
            f[-3] = (1-0.01*z[colNum['NU']])*z[colNum['NF']] - z[colNum['NE']] # NE = (1-0.01*NU)*NF
            f[-2] = z[colNum['GDP']]/z[colNum['NE']] -z[colNum['PDT']] # PDT = GDP/NE
            f[-1] = z[colNum['PWM']]/z[colNum['PDT']] -z[colNum['ULC']] # ULC = PWM/PDT
            return f
        root = fsolve(system,guess)  # method='hybr'
        # roots = newton_krylov(system,guess)
        root_rowdata.append(root)
        root_indexs.append(period)

    predictDf = pd.DataFrame(root_rowdata,columns=endogs,index=root_indexs)
    train_pred = predictDf.copy() #備用
    
    ############ 3. testing set predict ################
    testEndPeriod =  pd.Period(testEnd, 'Q-DEC')
    testPeriodRange = pd.period_range(start = trainEndPeriod+1,end =testEndPeriod)
    for period in testPeriodRange:
        print(period,end=',')
        allEqu = pd.DataFrame()
        for regNum,variables in enumerate(variable_2D):
            equDf,exogs =train_equDfs[regNum],all_exogs[regNum]
            # part d: 將 part c的外生變數之係數*真實值與常數項 (估計誤差不納入計算)，與內生變數併入 equDf2
            ### 外生變數暫時直接給定真實值(from data)###
        #     print(exogs)
            const = equDf['const'].values[0] + (data[exogs].loc[period]*equDf[exogs]).T.sum()  # float
            ### 納入residual的話聯立解會完全等於真實值，用來檢查過程有無錯誤
    #         residual = train_ols_results[regNum].resid.loc[period] # float
    # #         print(regNum,'resid into const')
    #         const = const + residual
            cols = []
            for c in equDf.columns:
                if c not in exogs and c != 'const':
                    cols.append(c)
            equDf2 = equDf[cols].copy()
            equDf2['const'] = const # equDf2 = 內生變數相關 (例如:年增率,差分,lag期)的係數跟方程式的常數項
        #     print('外生乘係數加進常數項後剩的columns:',equDf2.columns)
            # part e:轉換成當期函數
            # 找出差分項，再將lag期真實值乘係數加進const，ex: d1_xxxxx
        #     print('差分項:',end=' ')
            solvePeriod,difName,difNum=period,None,None
            difs = []
            if difName==None: # 自動找出差分項的名稱
                for name in equDf2.columns:
                    if re.search('d\d_',name):
                        difs.append(name)
            # else: # 有給定差分項變數名稱時
            #     difs.append(difName)
            for difName in difs:
                result = re.search('d(\d)_',difName)
                if result: 
                    difNum= int(result.group(1))
                    ori_x = difName.split(result.group())[-1]
        #             print(ori_x,difNum,end=',')
            #     else: #  其他非照規則(d1_)的自定義差分項，目前沒有
            #         ori_x = difName
            #         if difNum == None:
            #             difNum=1
                lagValue= predictDf.loc[solvePeriod-difNum][ori_x] * equDf2[difName].values #用 in-sample-pre的值
                equDf2['const'] = equDf2['const'].values - lagValue
                if ori_x in equDf2.columns:
        #             print(f'將同為{ori_x}的係數相加')
                    equDf2[ori_x] = equDf2[ori_x].values + equDf2[difName].values
                    equDf2.pop(difName)
                else:
        #             print(f'原{difName}改名為{ori_x}')
                    equDf2.rename(columns={difName:ori_x},inplace=True)

            # 找出lag項真實值乘係數加進const 
            # def equDf2_adj_lag(self,equDf2,solvePeriod):
            solvePeriod= period
        #     print('lag項:',end=' ')
            for name in equDf2.columns:
                result = re.search('(\w*)\(-(\d)\)',name)
                if result:
                    ori_x = result.group(1)
                    lagNum = int(result.group(2))
        #             print(ori_x,lagNum,end=',')
                    lagValue = predictDf.loc[solvePeriod-lagNum][ori_x] * equDf2[name].values #用 in-sample-pre的值
                    equDf2['const'] = equDf2['const'].values + lagValue
                    equDf2.pop(name)

            # 找出年增率處理成當期
        #     print('\n年增率項:',end=' ')
            for name in equDf2.columns:
                if 'pch_' in name:
                    ori_x = name.split('pch_')[1]
        #             print(name,'->',ori_x,end=', ')
                    equDf2['const'] = equDf2['const'].values-equDf2[name]
                    equDf2[name] = equDf2[name].values/predictDf.loc[solvePeriod-4][ori_x] # coef/y(-4) #用 in-sample-pre的值
                    # 若該年增率的內生變數剛好是這條回歸的y時，當pch_y轉成y後的係數coef/y(-4)可與係數為-1的y係數相加
                    if ori_x in equDf2.columns:
        #                 print(f'將同為{ori_x}的係數相加')
                        equDf2[ori_x] = equDf2[ori_x].values + equDf2[name].values
                        equDf2.pop(name)
                    else:
        #                 print(f'原{name}改名為{ori_x}')
                        equDf2.rename(columns={name:ori_x},inplace=True)
                    break

        #     print(f'\n處理完的columns:{equDf2.columns}')
            allEqu=pd.concat([allEqu,equDf2])

        # equ: CF + CO + IBF + J + X - M - GDP + const = 0  ( const=IG+IPC+CG )
        equ = pd.DataFrame(gdpequs.loc[period-1]).T
        allEqu = pd.concat([allEqu,equ])
        allEqu = allEqu[endogs+['const']]
        # part f: 猜該期真實值 解聯立
        guess = endogDf.loc[period-1].values
        def system(z,allEqu=allEqu): 
            #     if allEqu.columns[-1] != 'const':
            #         n = list(allEqu.columns).index('const')
            #         allEqu = allEqu[list(allEqu.columns[:n])+list(allEqu.columns[n+1:])+['const']]
            colNum={} # colNum[內生變數名] = 內生變數的欄位順序
            for i,col in enumerate(allEqu.columns):
                colNum[col] = i
            f= np.zeros(allEqu.shape[0]+3) # 3個額外的函數就加3
            # 適用於每條方程式皆係數*value加常數項
            for i,equ in enumerate(allEqu.iloc):
                f[i] = (equ[:-1]*z).sum()+ equ['const']
            # 函數裡的變數剛好全是內生變數時(都有在allEqu)
            f[-3] = (1-0.01*z[colNum['NU']])*z[colNum['NF']] - z[colNum['NE']] # NE = (1-0.01*NU)*NF
            f[-2] = z[colNum['GDP']]/z[colNum['NE']] -z[colNum['PDT']] # PDT = GDP/NE
            f[-1] = z[colNum['PWM']]/z[colNum['PDT']] -z[colNum['ULC']] # ULC = PWM/PDT
            return f
        root = fsolve(system,guess)  # method='hybr'
        # roots = newton_krylov(system,guess)
        predictDf = pd.concat([predictDf,pd.DataFrame([root],columns=endogs,index=[period])],axis=0)
        
    return predictDf


# 舊的

In [ ]:
def quick_select(data,       # 放入未 partial out Lag.Y 的 data
                 endog,      # 內生變數在data的欄位名
                 testCut,    # 留資料的後幾期當作test set
                 CV_n,       # LassoCV的TimeSeriesSplit(n_splits= "n")
                 alphaRange, # 檢驗alpha的範圍: np.logspace(-1,3,num=100)
                 partialOutAR1 = True, # 是否要partial out AR(1) 的資料丟入Lasso
                 showFitR2 =True,
                 showShrink=False
                ):
    """return L,S,df,ols_results 說明:
    L: lasso selected variable
    S: stepwise selected variable after lasso
    df: OLS note of S
    ols_results: 跑train期間OLS, y=endog, X= endog(-1)+ S
    """
    print('#'*100+f'\nendog= {endog} ,cv=TimeSeriesSplit(n_splits={CV_n})----------------------------------------------------\n')
    N = data.shape[0]
    if partialOutAR1:
        x1 = data[endog+'(-1)'].values.reshape(N,1)
        M1 = np.eye(N)-x1@inv(x1.T@x1)@x1.T # M1 = I-X1(X1'X1)^{-1}X1'
        y = M1@data[endog].values # M1y
        x = data.copy()
        x.drop(columns=[endog,'d1_'+endog,endog+'(-1)'],axis=0,inplace=True)
        x = M1@x
    else:
        print( 'not partial out AR(1) !')
        y = data[endog].values
        x = data.copy()
        x.drop(columns=[endog,'d1_'+endog],axis=0,inplace=True)
        
    cut=testCut
    x_train,x_test = x.iloc[:(N-cut)],x.iloc[(N-cut):]
    y_train,y_test = np.split(y,[N-cut])
#     print(x.shape , y.shape,x_train.shape,x_test.shape,y_train.shape,y_test.shape)
    print('train last period:',data.iloc[x_train.index[-1]].name,
          'test last period',data.iloc[x_test.index[-1]].name)
    n=CV_n
    alphas_grid = np.logspace(alphaRange[0], alphaRange[1], num = 200) # 範圍放比較大的，故使用200個點
    lasso_cv = linear_model.LassoCV(alphas = alphas_grid, fit_intercept = True, normalize = True, max_iter = 100000,
                                    tol = 0.0001, cv = TimeSeriesSplit(n_splits=n))
    lasso_cv_fitted = lasso_cv.fit(x_train, y_train)
    print('best alpha: ',lasso_cv_fitted.alpha_)
    best_alpha = lasso_cv_fitted.alpha_
    k = lasso_cv_fitted.coef_[lasso_cv_fitted.coef_!=0].shape[0]
    print('lasso選進',k,'個')
    L = x_train.columns[lasso_cv_fitted.coef_!=0]
    print('lasso選出的變數集合L:',L)
    fig, ax1 = plt.subplots()
    ax1.plot(lasso_cv_fitted.alphas_,lasso_cv_fitted.mse_path_, ':')
    ax1.plot(lasso_cv_fitted.alphas_, lasso_cv_fitted.mse_path_.mean(axis = -1),
             color='green',marker='o',markersize=3.5)
    ax1.set_xlabel(xlabel='alpha',size=13,color='black',rotation=10)
    ax1.set_ylabel(ylabel='mse',size=20,color='black',rotation=10,position=(0,1))
    plt.axvline(lasso_cv_fitted.alpha_, linestyle = '--', color = 'k', label = 'alpha: CV estimate')
    ax1.set_xscale('log')
    plt.show()
    
    y_hat_train = lasso_cv_fitted.predict(x_train)
    y_hat_test = lasso_cv_fitted.predict(x_test)
    if showFitR2:
        print('Performance on Training Data')
        print('  + RMSE: %.8f' % np.sqrt(mean_squared_error(y_train, y_hat_train)))
        # print('  + mean squared error: %.4f' % mean_squared_error(y_train, y_hat_train))
        print('  + R squares: %.8f' % r2_score(y_train, y_hat_train))
        print('Performance on Testing Data')
        print('  + RMSE: %.8f' % np.sqrt(mean_squared_error(y_test, y_hat_test)))
        print('  + R squares: %.8f' % r2_score(y_test, y_hat_test))
        print('\n')
    if showShrink:
        print(f'\nendog={endog}--coef shrinkage:')
        alpha100 = []
        for i,num in enumerate(alphas_grid):
            alpha100.append((num,num+0.000000000001))
        coefs = []
        klist = []
        for a in alpha100:
            lasso_cv = linear_model.LassoCV(alphas = a, fit_intercept = True, normalize = True, max_iter = 100000,
                                        tol = 0.001, cv = TimeSeriesSplit(n_splits=n))
            lasso_cv_fitted = lasso_cv.fit(x_train,y_train)
            coefs.append(lasso_cv_fitted.coef_)
            k = lasso_cv_fitted.coef_[lasso_cv_fitted.coef_!=0].shape[0]
            klist.append(k)

        fig, ax1 = plt.subplots()
        ax1.plot([a[0] for a in alpha100],coefs)
        # ax1.set_ybound(-100,100)
        plt.axvline(best_alpha, linestyle = '--', color = 'k', label = 'alpha: CV estimate')
        ax1.set_xscale('log')
        plt.show()

    smpl = x_train[L].copy()
    smpl[endog] = y_train.copy()
    S = stepwise_selected(data=smpl,response=endog,backward=True)
    print('stepwise選出的變數集合S:\n',S,'\n')
    df = show_ols_coef(X = x_train[S],y = y_train,indexName=f'endog={endog}')
    
    # "內生變數落後期"放進回歸自變數
    X = data[[f'{endog}(-1)']+S].iloc[:(N-cut)]
    X = sm.add_constant(X)
    ols_results = sm.OLS(data[endog].iloc[:(N-cut)],X).fit()
#     ols_results.save(f'../reg_pickle/{datetime.datetime.today().date()}{endog}.pickle')
    
    return L,S,df,ols_results

def macro_model6(data,sele_results,regressor_column,trainStart,trainEnd,testEnd):
    partialout_on =True
    trainStartPeriod = pd.Period(trainStart, 'Q-DEC')
    trainEndPeriod = pd.Period(trainEnd, 'Q-DEC')
    # 新增函數的內生變數名稱
    endogs = list(sele_results.index)+['GDP','NE','PDT','ULC']
    print('所有內生:',endogs)
    # train期間內每條回歸的smpl DataFrame、係數方程式 DataFrame、外生變數名 list
    trainPeriodRange = pd.period_range(start = trainStartPeriod,end =trainEndPeriod)
    train_smpls =[] # 每條迴歸的dataFrame
    train_ols_results=[] # 回頭檢查回歸用
    train_equDfs = [] # 每條迴歸的coef表示成函數 f(y,y(-1),exogs)=0 的dataFrame
    all_exogs = [] # 存出所有外生變數名稱
    # variable_2D = [ [y ,y(-1) , exogs],[y ,y(-1) , exogs],... ]
    variable_2D=[]
    for row in sele_results.iloc:
        if partialout_on:
            variable_2D.append([row.name,f'{row.name}(-1)']+row[regressor_column])
        else:
            variable_2D.append([row.name]+row[regressor_column])
    # regNum,variables = 0,variable_2D[0]
    for regNum,variables in enumerate(variable_2D):
    #     smpl = s.smpl_all(variables).loc[:trainEndPeriod] #最大樣本期間
        smpl = data[variables].loc[:trainEndPeriod].dropna()
        train_smpls.append(smpl)
        yName = smpl.columns[0]
        y = smpl[yName] # y固定為第一個元素
        X = smpl.copy()
        X.pop(yName)
        X = sm.add_constant(X)
        ols_results = sm.OLS(y, X).fit()
        train_ols_results.append(ols_results)
        params_df = pd.DataFrame(data=ols_results.params,columns=[yName])
        ###
        # part b: 將回歸的 y 放在 columns 並定義係數為 -1 (似等號另一邊為 0 ) 
        equDf = params_df.T.copy()
        equDf[params_df.columns[0]] = -1
        equDf.set_axis( [regNum] ,inplace=True)
        train_equDfs.append(equDf)
        # c: 存出回歸中哪些變數與聯立解的內生變數無關的list，方便下面自動執行外生係數乘
        exogs=[]
        for i,name in enumerate( a.myVariableName(variables) ):
            if name not in set(endogs):
                exogs.append(variables[i])
    #     print(sele_results.iloc[regNum].name,'\n',sele_results[regressor_column].iloc[regNum])
    #     print(exogs,'\n#################################################')
        all_exogs.append(exogs)

    ## 2. train期間聯立解存到 predictDf
    # 新增聯立解用的一條方程式:
    # CF + CO + IBF + IG + IPC + J + CG + X - M = GDP
    # CF + CO + IBF + J + X - M - GDP + const = 0  ( const=IG+IPC+CG )
    gdpdf = s.smpl_all(['CF','CO','IBF','IG','IPC','J','CG','EX','M','GDP'])
    gdpequs = pd.DataFrame([[1,1,1,1,1,-1,-1,1]],columns=['CF','CO','IBF','J','EX','M','GDP','const'],index=gdpdf.index)
    gdpequs['const'] = gdpdf[['IG','IPC','CG']].T.sum() # 每期的coef跟const
    # 已確認相等(gdpdf[gdpequs.columns[:-1]]*gdpequs[gdpequs.columns[:-1]]).T.sum()*(-1) == gdpequs['const']

    root_rowdata=[]
    root_indexs=[]
    endogDf = data[endogs]
    # 從 train第一期聯立解到最後一期的rowdata存進 predictDf DataFrame
    for period in trainPeriodRange:
        # period = trainPeriodRange[0]
        print(period,end=',')
        allEqu = pd.DataFrame()
        for regNum,variables in enumerate(variable_2D):
    #         print(regNum,variables)
            smpl,equDf,exogs = train_smpls[regNum],train_equDfs[regNum],all_exogs[regNum]
            # part d: 將 part c的外生變數之係數*真實值與常數項 (估計誤差不納入計算)，與內生變數併入 equDf2
            const = equDf['const'].values[0] + (smpl[exogs].loc[period]*equDf[exogs]).T.sum()  # float
            ### 納入residual的話聯立解會完全等於真實值，用來檢查過程有無錯誤
    #         residual = train_ols_results[regNum].resid.loc[period] # float
    #         const = const + residual
    #         print(regNum,'resid into const')
            cols = []
            for c in equDf.columns:
                if c not in exogs and c != 'const':
                    cols.append(c)
            equDf2 = equDf[cols].copy()
            equDf2['const'] = const # equDf2 = 內生變數相關 (例如:年增率,差分,lag期)的係數跟方程式的常數項
    #         print('外生乘係數加進常數項後剩的columns:',equDf2.columns)
            # part e:轉換成當期函數

            # 找出差分項，再將lag期真實值乘係數加進const，ex: d1_xxxxx
            # def equDf2_adj_dif(self,equDf2,solvePeriod,difName=None,difNum=None) 
            solvePeriod,difName,difNum=period,None,None #  difName = 'd1_RMIBON', difName = None : 自動搜索
            difs = []
            if difName==None: #找出差分項的名稱
                for name in equDf2.columns:
                    if re.search('d\d_',name):
                        difs.append(name)
            else:
                difs.append(difName)
    #         print(f'差分項:{difs}')
            for difName in difs:
                result = re.search('d(\d)_',difName)
                if result:
                    difNum= int(result.group(1))
                    ori_x = difName.split(result.group())[-1]
                else: #  其他非照規則(d1_)的自定義差分項
                    ori_x = difName
                    if difNum == None:
                        difNum=1
    #             print(ori_x,difNum,end=',')
                df = s.smpl_all([ori_x])
                lagValue= df.loc[solvePeriod-difNum].values * equDf2[difName].values
                equDf2['const'] = equDf2['const'].values - lagValue
                if ori_x in equDf2.columns:
    #                 print(f'將同為{ori_x}的係數相加')
                    equDf2[ori_x] = equDf2[ori_x].values + equDf2[difName].values
                    equDf2.pop(difName)
                else:
            #         print(f'原{difName}改名為{ori_x}')
                    equDf2.rename(columns={difName:ori_x},inplace=True)
            # 找出lag項真實值乘係數加進const 
            # def equDf2_adj_lag(self,equDf2,solvePeriod):
            solvePeriod= period
    #         print('\nlag項:',end=' ')
            for name in equDf2.columns:
                result = re.search('(\w*)\(-(\d)\)',name)
                if result:
                    x = result.group(1)
                    lagNum = int(result.group(2))
    #                 print(x,lagNum,end=',')
                    df = s.smpl_all([x])
                    lagValue = df.loc[solvePeriod-lagNum].values * equDf2[name].values
                    equDf2['const'] = equDf2['const'].values + lagValue
                    equDf2.pop(name)

            # 找出年增率處理成當期
    #         print('\n年增率項:',end=' ')
            for name in equDf2.columns:
                if 'pch_' in name:
                    ori_x = name.split('pch_')[1]
    #                 print(name,'->',ori_x,end=', ')
                    equDf2['const'] = equDf2['const'].values-equDf2[name]
                    df= s.smpl_all([ori_x])
                    equDf2[name] = equDf2[name].values/df.loc[solvePeriod-4].values # coef/y(-4)
                    # 若該年增率的內生變數剛好是這條回歸的y時，當pch_y轉成y後的係數coef/y(-4)可與係數為-1的y係數相加
                    if ori_x in equDf2.columns:
    #                     print(f'將同為{ori_x}的係數相加')
                        equDf2[ori_x] = equDf2[ori_x].values + equDf2[name].values
                        equDf2.pop(name)
                    else:
    #                     print(f'原{name}改名為{ori_x}')
                        equDf2.rename(columns={name:ori_x},inplace=True)
    #         print(f'\n處理完的columns:{equDf2.columns}')
            allEqu=pd.concat([allEqu,equDf2])
        # equ: CF + CO + IBF + J + X - M - GDP + const = 0  ( const=IG+IPC+CG )
        equ = pd.DataFrame(gdpequs.loc[period]).T
        allEqu = pd.concat([allEqu,equ])

        allEqu = allEqu[endogs+['const']]
        # part f: 猜該期真實值 解聯立
        guess = endogDf.loc[period].values
        ###
        def system(z,allEqu=allEqu): 
        #     if allEqu.columns[-1] != 'const':
        #         n = list(allEqu.columns).index('const')
        #         allEqu = allEqu[list(allEqu.columns[:n])+list(allEqu.columns[n+1:])+['const']]
            colNum={} # colNum[內生變數名] = 內生變數的欄位順序
            for i,col in enumerate(allEqu.columns):
                colNum[col] = i
            f= np.zeros(allEqu.shape[0]+3) # 3個額外的函數就加3
            # 適用於每條方程式皆係數*value加常數項
            for i,equ in enumerate(allEqu.iloc):
                f[i] = (equ[:-1]*z).sum()+ equ['const']
            # 函數裡的變數剛好全是內生變數時(都有在allEqu)
            f[-3] = (1-0.01*z[colNum['NU']])*z[colNum['NF']] - z[colNum['NE']] # NE = (1-0.01*NU)*NF
            f[-2] = z[colNum['GDP']]/z[colNum['NE']] -z[colNum['PDT']] # PDT = GDP/NE
            f[-1] = z[colNum['PWM']]/z[colNum['PDT']] -z[colNum['ULC']] # ULC = PWM/PDT
            return f
        root = fsolve(system,guess)  # method='hybr'
        # roots = newton_krylov(system,guess)
        root_rowdata.append(root)
        root_indexs.append(period)

    predictDf = pd.DataFrame(root_rowdata,columns=endogs,index=root_indexs)
    train_pred = predictDf.copy() #備用
    
    ## 3. testing set predict
    testEndPeriod =  pd.Period(testEnd, 'Q-DEC')
    testPeriodRange = pd.period_range(start = trainEndPeriod+1,end =testEndPeriod)
    for period in testPeriodRange:
        print(period,end=',')
        allEqu = pd.DataFrame()
        for regNum,variables in enumerate(variable_2D):
            equDf,exogs =train_equDfs[regNum],all_exogs[regNum]
            # part d: 將 part c的外生變數之係數*真實值與常數項 (估計誤差不納入計算)，與內生變數併入 equDf2
            ### 外生變數暫時直接給定真實值(from data)###
        #     print(exogs)
            const = equDf['const'].values[0] + (data[exogs].loc[period]*equDf[exogs]).T.sum()  # float
            ### 納入residual的話聯立解會完全等於真實值，用來檢查過程有無錯誤
    #         residual = train_ols_results[regNum].resid.loc[period] # float
    # #         print(regNum,'resid into const')
    #         const = const + residual
            cols = []
            for c in equDf.columns:
                if c not in exogs and c != 'const':
                    cols.append(c)
            equDf2 = equDf[cols].copy()
            equDf2['const'] = const # equDf2 = 內生變數相關 (例如:年增率,差分,lag期)的係數跟方程式的常數項
        #     print('外生乘係數加進常數項後剩的columns:',equDf2.columns)
            # part e:轉換成當期函數
            # 找出差分項，再將lag期真實值乘係數加進const，ex: d1_xxxxx
        #     print('差分項:',end=' ')
            solvePeriod,difName,difNum=period,None,None
            difs = []
            if difName==None: # 自動找出差分項的名稱
                for name in equDf2.columns:
                    if re.search('d\d_',name):
                        difs.append(name)
            # else: # 有給定差分項變數名稱時
            #     difs.append(difName)
            for difName in difs:
                result = re.search('d(\d)_',difName)
                if result: 
                    difNum= int(result.group(1))
                    ori_x = difName.split(result.group())[-1]
        #             print(ori_x,difNum,end=',')
            #     else: #  其他非照規則(d1_)的自定義差分項，目前沒有
            #         ori_x = difName
            #         if difNum == None:
            #             difNum=1
                lagValue= predictDf.loc[solvePeriod-difNum][ori_x] * equDf2[difName].values #用 in-sample-pre的值
                equDf2['const'] = equDf2['const'].values - lagValue
                if ori_x in equDf2.columns:
        #             print(f'將同為{ori_x}的係數相加')
                    equDf2[ori_x] = equDf2[ori_x].values + equDf2[difName].values
                    equDf2.pop(difName)
                else:
        #             print(f'原{difName}改名為{ori_x}')
                    equDf2.rename(columns={difName:ori_x},inplace=True)

            # 找出lag項真實值乘係數加進const 
            # def equDf2_adj_lag(self,equDf2,solvePeriod):
            solvePeriod= period
        #     print('lag項:',end=' ')
            for name in equDf2.columns:
                result = re.search('(\w*)\(-(\d)\)',name)
                if result:
                    ori_x = result.group(1)
                    lagNum = int(result.group(2))
        #             print(ori_x,lagNum,end=',')
                    lagValue = predictDf.loc[solvePeriod-lagNum][ori_x] * equDf2[name].values #用 in-sample-pre的值
                    equDf2['const'] = equDf2['const'].values + lagValue
                    equDf2.pop(name)

            # 找出年增率處理成當期
        #     print('\n年增率項:',end=' ')
            for name in equDf2.columns:
                if 'pch_' in name:
                    ori_x = name.split('pch_')[1]
        #             print(name,'->',ori_x,end=', ')
                    equDf2['const'] = equDf2['const'].values-equDf2[name]
                    equDf2[name] = equDf2[name].values/predictDf.loc[solvePeriod-4][ori_x] # coef/y(-4) #用 in-sample-pre的值
                    # 若該年增率的內生變數剛好是這條回歸的y時，當pch_y轉成y後的係數coef/y(-4)可與係數為-1的y係數相加
                    if ori_x in equDf2.columns:
        #                 print(f'將同為{ori_x}的係數相加')
                        equDf2[ori_x] = equDf2[ori_x].values + equDf2[name].values
                        equDf2.pop(name)
                    else:
        #                 print(f'原{name}改名為{ori_x}')
                        equDf2.rename(columns={name:ori_x},inplace=True)
                    break

        #     print(f'\n處理完的columns:{equDf2.columns}')
            allEqu=pd.concat([allEqu,equDf2])

        # equ: CF + CO + IBF + J + X - M - GDP + const = 0  ( const=IG+IPC+CG )
        equ = pd.DataFrame(gdpequs.loc[period-1]).T
        allEqu = pd.concat([allEqu,equ])
        allEqu = allEqu[endogs+['const']]
        # part f: 猜該期真實值 解聯立
        guess = endogDf.loc[period-1].values
        def system(z,allEqu=allEqu): 
            #     if allEqu.columns[-1] != 'const':
            #         n = list(allEqu.columns).index('const')
            #         allEqu = allEqu[list(allEqu.columns[:n])+list(allEqu.columns[n+1:])+['const']]
            colNum={} # colNum[內生變數名] = 內生變數的欄位順序
            for i,col in enumerate(allEqu.columns):
                colNum[col] = i
            f= np.zeros(allEqu.shape[0]+3) # 3個額外的函數就加3
            # 適用於每條方程式皆係數*value加常數項
            for i,equ in enumerate(allEqu.iloc):
                f[i] = (equ[:-1]*z).sum()+ equ['const']
            # 函數裡的變數剛好全是內生變數時(都有在allEqu)
            f[-3] = (1-0.01*z[colNum['NU']])*z[colNum['NF']] - z[colNum['NE']] # NE = (1-0.01*NU)*NF
            f[-2] = z[colNum['GDP']]/z[colNum['NE']] -z[colNum['PDT']] # PDT = GDP/NE
            f[-1] = z[colNum['PWM']]/z[colNum['PDT']] -z[colNum['ULC']] # ULC = PWM/PDT
            return f
        root = fsolve(system,guess)  # method='hybr'
        # roots = newton_krylov(system,guess)
        predictDf = pd.concat([predictDf,pd.DataFrame([root],columns=endogs,index=[period])],axis=0)
        
    return predictDf

In [ ]:
#     def line_ols_results(self,ols_results,seriesActual=None,resid_ybound=True,name='untitled'):
#         """ 改用下面的line_fit即可"""
#         t = ols_results.resid.index.to_timestamp()
#         dataResid = ols_results.resid.values
#         dataFit = ols_results.fittedvalues.values
#         lineZero = [0]*len(t)

#         fig, ax1 = plt.subplots()
#         ax1.plot(t, dataResid, color='green',label='residual',marker='o',markersize=3.5)
#         ax1.plot(t,lineZero,color='black',linestyle=':')
#         ax1.set_xlabel(xlabel='time(Q)',size=13,color='black',rotation=10)
#         ax1.set_ylabel(ylabel='resid',size=20,color='green',rotation=0,position=(0,0.95))
#         ax1.tick_params(axis='x', labelcolor='black',labelrotation=10,labelsize=13)
#         ax1.tick_params(axis='y', labelcolor='green',labelsize=13)

#         ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
#         ax2.plot(t, dataFit, color='r')
#         if type(seriesActual) == pd.core.series.Series:
#             rmse = np.sqrt( np.mean( (seriesActual-dataFit)**2 ) )
#             ax1.set_title(label='rmse = {}'.format(rmse),fontdict={'fontsize':20})
#             ax2.plot(t, seriesActual, color='b')
#         if resid_ybound:
#             # 取得上下界的值
#             y2min,y2max = ax2.get_ybound()
#             up_bound = (y2max-y2min)/2
#             # 設定顯示哪些刻度，此例切8等分
#             ax2.set_yticks([x for x in np.arange(y2min,y2max,up_bound*2/8)])
#             # 設定左y軸全距 = 右y軸全距
#             ax1.set_ybound(-up_bound,up_bound)
#             ax1.set_yticks(np.arange(-up_bound,up_bound,up_bound*2/8)) # 誤差刻度也是切8等分
#         ax2.tick_params(axis='y', labelcolor='b',labelsize=13)
#         ax2.legend(labels=['fitted_'+name,'actual_'+name],loc=0,fontsize=20,shadow=True)

#         fig.tight_layout()  # otherwise the right y-label is slightly clipped
#     #   plt.savefig(fname='../report/0111/{}_{}.png'.format(regNum,variable_2D[regNum][0]))
#         plt.show()

In [ ]:
# def stepwise_selected(data,response,backward=2):
#     """choose one type:
#     0. only forward
#     1. backward by check almost each set after pick one
#     2. backward by remove lowest contribute selected
#     choose score:
#     0. adjusted R-squared
#     #1. AIC
#     #2. F test
#     """
#     remaining = [x for x in data.columns]
#     remaining.remove(response)
#     selected = []
#     current_score, best_new_score = 0.0, 0.0
#     i=0
#     while remaining and current_score == best_new_score:
#         i=i+1
#         scores_with_candidates = []
#         for candidate in remaining:
#             formula = "{} ~ {} + 1".format(response,
#                             " + ".join(["Q('"+x+"')" for x in selected] + ["Q('"+candidate+"')"]))
#             score = smf.ols(formula, data).fit().rsquared_adj
#             scores_with_candidates.append((score, candidate))
#         scores_with_candidates.sort()
#         best_new_score, best_candidate = scores_with_candidates.pop() #最高分的
#     #     print(best_new_score, best_candidate,end=',')
#         if current_score < best_new_score:
#             remaining.remove(best_candidate)
#             selected.append(best_candidate)
#             current_score = best_new_score
#             # backward 1: 當前當選的變數群裡，從第一個開始被候選池變數替換，有更好表現則選入，且將原當選變數丟回候選池。
#             if backward==1:
#                 if i ==1: # 抓進第一個變數後不需向後
#                     continue
#                 for j,elected in enumerate(selected):
#                     otherSelected = selected.copy()
#                     otherSelected.remove(elected)
#                     scores_with_candidates2 = []
#                     for candidate in remaining:
#                         formula = "{} ~ {} + 1".format(response,
#                                         " + ".join(["Q('"+x+"')" for x in otherSelected] + ["Q('"+candidate+"')"]))
#                         score2 = smf.ols(formula, data).fit().rsquared_adj
#                         scores_with_candidates2.append((score2, candidate))
#                     scores_with_candidates2.sort()
#                     best_new_score2, best_candidate2 = scores_with_candidates2.pop()
#             #         print('向後:',best_new_score2, best_candidate2,end=',')
#                     if current_score < best_new_score2:
#                         print('第 {} round:原組合為{}'.format(i,selected))
#                         remaining.remove(best_candidate2)
#                         remaining.append(elected) # 原選入被淘汰的變數丟回候選池
#                         selected[j] = best_candidate2 # 原選入被淘汰的變數換成這次當選的
#                         print('{}替換成{}\n'.format(elected,best_candidate2))
#                         current_score = best_new_score2
#                         best_new_score = best_new_score2 # Let whlie True
#             # backward 2: 當前當選的變數群，用 full model - reduce model的 adjusted R-squared 得各變數的貢獻度，
#             #             若最低貢獻不是最後一次選進的變數則丟回候選池。
#             elif backward == 2:
#                 if i <=2: # 三個變數再開始向後
#                     continue
#                 contribute_with_electeds = []
#                 for j,elected in enumerate(selected):
#                     contribute_elected = elected
#                     otherSelected = selected.copy()
#                     otherSelected.remove(elected)
#                     formula = "{} ~ {} + 1".format(response,
#                                 " + ".join(["Q('"+x+"')" for x in otherSelected]))
#                     score2 = smf.ols(formula, data).fit().rsquared_adj
#                     contribute_score = current_score - score2
#                     contribute_with_electeds.append((contribute_score,contribute_elected))
#                 contribute_with_electeds.sort()
#                 lowest_score_elected = contribute_with_electeds[0][1]
#                 if lowest_score_elected != selected[-1]:
#                     print('第 {} round:剔除{}'.format(i,lowest_score_elected))
#                     remaining.append(lowest_score_elected)
#                     selected.remove(lowest_score_elected)
#         else:
#             if backward == 2:
#                 best_new_score = 0.0 
#                 # 因最近一次的剔除跟選入的變數會是同一個，導致best剛好等於current，所以要強制停止whlie
#             print('第{} round end \n 選進{}個變數'.format(i,len(selected)))
#     return selected

In [ ]:
# def system5(z,allEqu=allEqu):
#     f = np.zeros(5)
#     f[0] = allEqu.loc[0]['RMIBON']*z[0] + allEqu.loc[0]['IR']*z[1] + allEqu.loc[0]['const']
#     f[1] = allEqu.loc[1]['RMIBON']*z[0] + allEqu.loc[1]['IR']*z[1] + allEqu.loc[1]['log_ADRESERVE$']*math.log(z[2]) + allEqu.loc[1]['const']
#     f[2] = allEqu.loc[2]['RMIBON']*z[0] + allEqu.loc[2]['log_ADRESERVE$']*math.log(z[2]) + allEqu.loc[2]['log_DEPOSIT']*math.log(z[3]) + allEqu.loc[2]['const']
#     f[3] = allEqu.loc[3]['log_DEPOSIT']*math.log(z[3]) + allEqu.loc[3]['CPIZF']*z[4] + allEqu.loc[3]['const']
#     f[4] = allEqu.loc[4]['RMIBON']*z[0] + allEqu.loc[4]['CPIZF']*z[4] + allEqu.loc[4]['const']
#     return f

In [ ]:
#     def model_to_params_df(self,ols_model):
#         y_name = ols_model.endog_names
#         ols_results = ols_model.fit()
#         params_df = pd.DataFrame(data=ols_results.params,columns=[y_name])
#         return params_df   
#     def smpl_ols_params(self,variable_list):
#         smpl = selecter().smpl_all(variable_list)
#         res = self.smpl_ols_model(smpl)
#         res_df = self.model_to_params_df(res)
#         return smpl,res,res_df
    
#     def smpl_to_params_df(self,variable_list):
#         smpl = selecter().smpl_all(variable_list)
#         results = self.smpl_ols_model(smpl)
#         params_df = self.model_to_params_df(results)
#         return params_df        
#     def get_allParams(self,variable_2D):
#         all_params = pd.DataFrame()
#         for variable_list in variable_2D:
#             params_df = self.smpl_to_params_df(variable_list)
#             all_params = pd.concat([all_params,params_df],axis='columns')
#         return all_params
    
#     def allParams_to_equs(self,pat):
#         pat = pa.T.copy()
#         #常數項移到等號左邊變號
#         pat['const'] = pat['const'].apply(lambda x: -x)
#         #每條回歸的y移至等號右邊
#         yname = pat.index[0]
#         if yname not in pat.loc[yname].index:
#             pat[yname] = np.nan
#         for colname in pat.columns:
#             if pat[colname].name in pat[colname].index: 
#                 pat[colname][colname]=-1
#         #改index名稱
#         new_index = []
#         for i in range(pat.shape[0]):
#             new_index.append('equ{}'.format(i))
#         equs = pat.set_axis(new_index) #.fillna(0)
#         return equs
    
#     def smpls_firstPeriod(self,smpls):
#         """ 共同樣本期間的第一期"""
#         firstPeriod = pd.Period('1900Q1')
#         for df in smpls:
#             #print(df.index[0])
#             if df.index[0] > firstPeriod:
#                 firstPeriod = df.index[0]
#         print('共同樣本期間第一期為',firstPeriod)
#         return firstPeriod
#     def equs_to_gsEqus(self,results,equs,gsEndogNames,firstPeriod):
#         """ results為每一條回歸的ols_model的list 來自smpl_ols_params(self,variable_list)[1]
#         # 若要取消將resid_hat加總進常數項則參數設為results=None
#         firstPeriod= 0 or pd.Period('1982Q2') """
#         #取出外生變數係數df
#         gs_exog_names=[]
#         for x in equs.columns:
#             if x not in gsEndogNames:
#                 gs_exog_names.append(x)
#         ex_df=equs[gs_exog_names].copy()
#         #得每條外生變數之係數乘第一期真實值+const
#         for i in range(ex_df.shape[0]): #每條回歸
#             param = ex_df.iloc[i]
#             smpl = smpls[i].copy()
#             for ex in param.index: #回歸中每個外生變數
#                 if ex in smpl.columns:
#                     #print('外生:{},--係數:{},--firstPeriod值:{}'.format(ex,param[ex],smpl[ex][firstPeriod]) )
#                     param[ex] = -1*param[ex]*smpl[ex][firstPeriod] # 係數*真實值
#                     # 乘負號是因後面要視為移項到常數項去加總
#         # 將 resid_hat也加總進常數項
#         if results:
#             resids = []
#             for result in results:
#                 resids.append(result.fit().resid[firstPeriod])
#             ex_df['resid_hat'] = resids

#         gs_con = pd.DataFrame(ex_df.T.sum(),columns=['gs_con']) #常數項跟其他相乘項加總
#         #print(gs_con)
#         en_df = equs[gsEndogNames]
#         gsEqus = pd.concat([gs_con,en_df],axis='columns')
#         return gsEqus
    
#     def initialValue_real(self,gsEndog,firstPeriod):
#         initialValue = selecter().smpl_all(gsEndog).loc[firstPeriod] # + random.random()
#         return initialValue.values.round(4)
    
#     def gauss_seidel(self,param_matrix ,con_matrix ,
#                     initialValue = None,tol=1e-7,max_iteration=1000):
#         """A and b need numpy.array, A: initialize the matrix, b: initialize the RHS vector"""
#         ITERATION_LIMIT = 1000000
#         A = param_matrix
#         b = con_matrix
#         print("System of equations:")
#         for i in range(A.shape[0]):
#             row = ["{0:3g}*x{1}".format(A[i, j], j + 1) for j in range(A.shape[1])]
#             print("[{0}] = [{1:3g}]".format(" + ".join(row), b[i]))
        
#         if type(initialValue) != np.ndarray:
#             initialValue = np.zeros_like(con_matrix)
#             print('fuck')
                
#         x = initialValue
#         print(x)
#         for it_count in range(1, ITERATION_LIMIT):
#             x_new = np.zeros_like(x)
#             if it_count > max_iteration:
#                 print('NOT CONVERGE!')
#                 return
#             print("Iteration {0}: {1}".format(it_count, x))

#             for i in range(A.shape[0]):
#                 s1 = np.dot(A[i, :i], x_new[:i])
#                 s2 = np.dot(A[i, i + 1:], x[i + 1:])
#                 x_new[i] = (b[i] - s1 - s2) / A[i, i]
#             if np.allclose(x, x_new, tol):
#                 break
#             x = x_new    
#         print("Solution: {0}".format(x))
#         error = np.dot(A, x) - b
#         print("Error: {0}".format(error))
#         return x
    
#     def np_linear_solve(self,param_matrix,con_matrix):
# #         Ax = B  
#         #寫出係數矩陣 A
#         A = param_matrix
#         # 寫出常數矩陣 B
#         B = con_matrix
#         # 找出係數矩陣的反矩陣 A_inv
#         A_inv = np.linalg.inv(A)
#         # 將 A_inv 與 B 相乘，即可得到解答
#         sol = A_inv.dot(B)
#         return so

In [13]:
# IRC = C + IR + RMIBON - D2003 + AR(1)0.8645
# RMIBON = C + IR + @PCH(CPI) + LOG(PSTOCK) + ADRESERVE$ + e
# ADRESERVE$ = C + GDP$ + DEPOSIT + D(RMIBON) + AR(1)0.8190
# LOG(DEPOSIT) = -C + LOG(GDP) - D(LOG(GDP)) + D(IRC-@PCHY(CPIZF)) - @PCHY(STOCKTRADE) + AR(1)0.5753
# D(CPIZF) = -C - D(RMIBON) - NU + LOG(M2) + AR(1)-0.2726
# NU = C + NU(-4) - SALES/GDP + CHINAGDP - D2003 + AR(1)0.8201
# M2 = -C + M2(-1) + GDP + (IRC-IRCUS) + D(STOCKTRADE) + Q1 - Q2

# variable_2D = [
#     ['IRC','IR','RMIBON','D2003'],
#     ['RMIBON','IR','pch_CPI','log_PSTOCK','ADRESERVE$'],
#     ['ADRESERVE$','GDP$','DEPOSIT','d1_RMIBON'],
#     ['log_DEPOSIT','log_GDP','D(LOG(GDP))','D(IRC-@PCHY(CPIZF))','pch_STOCKTRADE'],
#     ['d1_CPIZF','d1_RMIBON','NU','log_M2'],
#     ['NU','NU(-4)','SALES/GDP','CHINAGDP','D2003'],
#     ['M2','M2(-1)','GDP','(IRC-IRCUS)','d1_STOCKTRADE','Q1','Q2'],
# ]

# lens = len(variable_2D)
# smpls=[None]*lens
# res=[None]*lens
# resDf=[None]*lens
# for i,variable_list in enumerate(variable_2D):
#     smpls[i],res[i],resDf[i] = a.smpl_ols_params(variable_list)

# for i,variable_list in enumerate(variable_2D):
#     i=i+1
#     globals()['smpl%s'%i],globals()['res%s'%i],globals()['res_df%s'%i] = a.smpl_ols_params(variable_list)

Wall time: 3.55 s


# ------------------------------------------------------

# 範例